<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=909faf527aed94ec88b8061a8289fa4b1156a3b56531355a7c95b491a417308a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 12:14:36
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.38 C
-------------------
Today PnL: -1.94 L (-1.38%)
Current PnL: -23.19 L (-15.45%)
CY Booked + Current PnL: -11.29 L (-7.52%)
-------------------
Total profit:  1.19 L
Total loss:  -24.38 L
-------------------
Total Booked + Current PnL: 15.69 L (12.75%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.6%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 90.39 L (65.36%)
Deployed:  1.23 C
Current:  1.38 C
CAGR/XIRR %: 6.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,98.29,53.0,M-SC,1.26,85425.0,-15352.0,15445.0,2.31,-15.23,18.08,0.09,245.0,-0.99,0.62,12.23,OX40N,NTT,DURABLES
18,CIPLA,1662.74,-20.47,53.0,X-LC,3.73,211775.0,7275.0,16031.0,-1.14,3.56,7.57,11.39,10.0,0.45,1.54,12.83,X40N,ATH,PHARMA
83,VOLTAS,1530.00,-0.80,52.0,X-MC,1.02,208050.0,16308.0,21450.0,0.62,8.51,10.31,19.69,99.0,0.76,1.52,15.88,XY25,NTT,AC
56,RELIANCE,1533.00,-14.04,48.0,X-LC,2.85,214890.0,4524.0,24261.0,0.18,2.15,11.29,13.68,37.0,0.19,1.57,19.03,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,40.0,X-LC,2.74,194769.0,-5369.0,26255.0,-0.24,-2.68,13.48,10.44,4.0,-0.20,1.42,2.73,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,98.29,53.0,M-SC,1.26,85425.0,-15352.0,15445.0,2.31,-15.23,18.08,0.09,245.0,-0.99,0.62,12.23,OX40N,NTT,DURABLES
34,ICICIPRULI,790.00,-20.77,52.0,X-MC,1.92,136278.0,490.0,42260.0,1.58,0.36,31.01,31.48,107.0,0.01,0.99,12.52,X40,ATH,INSURANCE
85,WIPRO,420.00,-12.43,54.0,M-LC,5.94,154002.0,3057.0,106400.0,1.33,2.03,69.09,72.51,53.0,0.03,1.12,8.17,XR,NTT,IT
66,SONACOMS,1006.00,-38.01,59.0,M-SC,12.98,77201.0,-23958.0,98848.0,1.24,-23.68,128.04,74.03,202.0,-0.24,0.56,8.86,AR,BTT,AUTO
33,ICICIGI,2260.25,-20.34,43.0,X-MC,1.47,137115.0,982.0,30138.0,1.06,0.72,21.98,22.86,91.0,0.03,1.00,13.43,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-53.17,12.0,X-LC,1.01,148031.0,-128492.0,238567.0,-38.28,-46.47,161.16,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
23,DEN,75.0,89.12,37.0,M-SC,3.87,52944.0,-31520.0,67054.0,-3.86,-37.32,126.65,42.07,237.0,-0.47,0.39,12.44,AR,NTT,ENTERTAINMENT
81,VALIANTORG,838.0,-280.29,24.0,H-SC,3.06,100300.0,-63305.0,184622.0,-3.44,-38.69,184.07,74.15,139.0,-0.34,0.73,28.57,XR,NTT,CHEMICALS
10,BANDHANBNK,400.0,-12.63,48.0,H-SC,2.96,218018.0,-60541.0,317979.0,-3.43,-21.73,145.85,92.42,151.0,-0.19,1.59,26.00,XY24,NTT,BANKS
63,SHALBY,327.0,1161.56,52.0,M-SC,22.89,171587.0,-9556.0,55028.0,-2.84,-5.28,32.07,25.10,235.0,-0.17,1.25,37.51,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.3,123940.0,604.0,131017.0,-2.50,0.49,105.71,106.72,119.0,0.0,0.90,23.98,AR,ATH,MISC
54,RECLTD,446.00,45.36,42.0,M-LC,6.2,203005.0,-165.0,42286.0,-0.54,-0.08,20.83,20.74,55.0,-0.0,1.48,5.46,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,104.20,56.0,H-SC,11.54,129087.0,-13392.0,32917.0,-1.91,-9.40,25.50,13.70,163.0,-0.41,0.94,52.36,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.29,53.0,M-SC,1.26,85425.0,-15352.0,15445.0,2.31,-15.23,18.08,0.09,245.0,-0.99,0.62,12.23,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,2.06,220824.0,-48843.0,85172.0,-0.14,-18.11,38.57,13.47,138.0,-0.57,1.61,12.28,XY24,NTT,PAINTS
17,CERA,9475.0,-24.11,38.0,H-SC,2.00,138483.0,-37420.0,79448.0,0.15,-21.27,57.37,23.89,149.0,-0.47,1.01,19.60,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-25.05,42.0,H-MC,7.03,101365.0,-30470.0,72699.0,-0.32,-23.11,71.72,32.03,98.0,-0.42,0.74,13.54,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,44.35,51.0,M-MC,12.65,228182.0,3220.0,164428.0,0.26,1.43,72.06,74.53,192.0,0.02,1.66,34.15,XY24,BTT,STEEL
31,HINDZINC,730.22,34.27,66.0,M-LC,8.54,219175.0,14099.0,98475.0,-0.34,6.87,44.93,54.89,52.0,0.14,1.60,32.63,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,58.0,M-LC,9.39,159066.0,18226.0,26262.0,0.45,12.94,16.51,31.59,66.0,0.69,1.16,42.86,XY24,NTT,BANKS
85,WIPRO,420.00,-12.43,54.0,M-LC,5.94,154002.0,3057.0,106400.0,1.33,2.03,69.09,72.51,53.0,0.03,1.12,8.17,XR,NTT,IT
11,BANKINDIA,190.00,-24.22,68.0,H-MC,12.35,195111.0,15303.0,97107.0,0.07,8.51,49.77,62.52,88.0,0.16,1.42,45.50,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-24.22,68.0,H-MC,12.35,195111.0,15303.0,97107.0,0.07,8.51,49.77,62.52,88.0,0.16,1.42,45.50,XR,NTT,BANKS
31,HINDZINC,730.22,34.27,66.0,M-LC,8.54,219175.0,14099.0,98475.0,-0.34,6.87,44.93,54.89,52.0,0.14,1.60,32.63,X5K,ATH,METALS
85,WIPRO,420.00,-12.43,54.0,M-LC,5.94,154002.0,3057.0,106400.0,1.33,2.03,69.09,72.51,53.0,0.03,1.12,8.17,XR,NTT,IT
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.30,123940.0,604.0,131017.0,-2.50,0.49,105.71,106.72,119.0,0.00,0.90,23.98,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-48.49,31.0,M-SC,3.39,87045.0,-4454.0,83206.0,-0.31,-4.87,95.59,86.07,223.0,-0.05,0.63,26.40,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.17,12.0,X-LC,1.01,148031.0,-128492.0,238567.0,-38.28,-46.47,161.16,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-9.01,28.0,X-MC,1.38,225498.0,-22233.0,97280.0,-1.87,-8.97,43.14,30.30,92.0,-0.23,1.64,3.12,X40,ATH,ELECTRICAL
51,QUESS,986.00,-48.21,29.0,X-SC,37.86,51246.0,-13760.0,164689.0,-1.89,-21.17,321.37,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
9,BAJAJHFL,181.50,-16.38,30.0,X-MC,5.51,176218.0,-24288.0,121449.0,-1.17,-12.11,68.92,48.45,90.0,-0.20,1.28,1.62,X40N,ATH,FINANCE
5,ASIANPAINT,3465.66,-19.07,31.0,X-LC,5.31,204283.0,-47485.0,100691.0,-0.90,-18.86,49.29,21.13,27.0,-0.47,1.49,9.88,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-4.54,39.0,H-LC,0.91,153450.0,-32645.0,80132.0,-0.28,-17.54,52.22,25.51,15.0,-0.41,1.12,13.08,AR,ATH,ELECTRICAL
72,TATAMOTORS,1065.00,-53.17,12.0,X-LC,1.01,148031.0,-128492.0,238567.0,-38.28,-46.47,161.16,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
83,VOLTAS,1530.00,-0.80,52.0,X-MC,1.02,208050.0,16308.0,21450.0,0.62,8.51,10.31,19.69,99.0,0.76,1.52,15.88,XY25,NTT,AC
28,HAVELLS,2069.16,-9.01,28.0,X-MC,1.38,225498.0,-22233.0,97280.0,-1.87,-8.97,43.14,30.30,92.0,-0.23,1.64,3.12,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.34,43.0,X-MC,1.47,137115.0,982.0,30138.0,1.06,0.72,21.98,22.86,91.0,0.03,1.00,13.43,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.86,51246.0,-13760.0,164689.0,-1.89,-21.17,321.37,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.17,12.0,X-LC,1.01,148031.0,-128492.0,238567.0,-38.28,-46.47,161.16,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-16.38,30.0,X-MC,5.51,176218.0,-24288.0,121449.0,-1.17,-12.11,68.92,48.45,90.0,-0.20,1.28,1.62,X40N,ATH,FINANCE
82,VBL,671.64,-21.70,38.0,X-LC,10.50,282060.0,-33782.0,146446.0,-0.08,-10.70,51.92,35.67,5.0,-0.23,2.06,1.68,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.42,36.0,X-MC,7.67,220650.0,-42715.0,156066.0,-0.66,-16.22,70.73,43.04,84.0,-0.27,1.61,2.44,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.86,51246.0,-13760.0,164689.0,-1.89,-21.17,321.37,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-41.14,34.0,X-SC,3.33,80267.0,-64893.0,140820.0,-1.16,-44.70,175.44,52.31,136.0,-0.46,0.59,7.44,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.29,34.0,X-SC,4.35,93707.0,-34963.0,80260.0,-1.74,-27.17,85.65,35.20,219.0,-0.44,0.68,7.55,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.49,45.0,X-SC,1.04,106280.0,4276.0,39653.0,-1.88,4.19,37.31,43.06,122.0,0.11,0.77,22.03,X40N,ATH,MISC
32,HONAUT,58357.33,-23.49,43.0,X-SC,2.24,107835.0,-20103.0,67235.0,-0.58,-15.71,62.35,36.84,143.0,-0.30,0.79,10.40,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.97,44.0,X-LC,12.29,278917.0,-67039.0,135275.0,-1.33,-19.38,48.50,19.73,1.0,-0.50,2.03,2.73,X40,ATH,IT
39,INFY,2275.00,-18.52,51.0,X-LC,7.50,317136.0,4248.0,167448.0,-0.29,1.36,52.80,54.87,3.0,0.03,2.31,8.10,X40,BTT,IT
41,ITC,452.00,-39.61,40.0,X-LC,2.74,194769.0,-5369.0,26255.0,-0.24,-2.68,13.48,10.44,4.0,-0.20,1.42,2.73,X40,NTT,FMCG
82,VBL,671.64,-21.70,38.0,X-LC,10.50,282060.0,-33782.0,146446.0,-0.08,-10.70,51.92,35.67,5.0,-0.23,2.06,1.68,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,40.0,H-LC,7.29,245544.0,-16075.0,135295.0,-0.62,-6.14,55.10,45.57,7.0,-0.12,1.79,4.48,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.72,81024.0,-32525.0,72573.0,-0.92,-28.64,89.57,35.27,268.0,-0.45,0.59,98.31,XR,NTT,HOTELS
48,MASFIN,398.61,-21.41,39.0,H-SC,9.91,89445.0,-8535.0,30134.0,-1.23,-8.71,33.69,22.05,152.0,-0.28,0.65,30.01,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,104.20,56.0,H-SC,11.54,129087.0,-13392.0,32917.0,-1.91,-9.40,25.50,13.70,163.0,-0.41,0.94,52.36,OX40N,NTT,BANKS
60,SAMMAANCAP,326.00,-184.09,64.0,M-SC,17.37,145179.0,-4041.0,148228.0,-0.68,-2.71,102.10,96.62,208.0,-0.03,1.06,57.12,XY25,NTT,FINANCE
68,SURYODAY,240.00,60.09,54.0,H-SC,10.49,146125.0,-32946.0,98678.0,-2.80,-18.40,67.53,36.71,135.0,-0.33,1.07,44.99,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-184.09,64.0,M-SC,17.37,145179.0,-4041.0,148228.0,-0.68,-2.71,102.10,96.62,208.0,-0.03,1.06,57.12,XY25,NTT,FINANCE
68,SURYODAY,240.00,60.09,54.0,H-SC,10.49,146125.0,-32946.0,98678.0,-2.80,-18.40,67.53,36.71,135.0,-0.33,1.07,44.99,XR,NTT,BANKS
31,HINDZINC,730.22,34.27,66.0,M-LC,8.54,219175.0,14099.0,98475.0,-0.34,6.87,44.93,54.89,52.0,0.14,1.60,32.63,X5K,ATH,METALS
67,STARHEALTH,761.00,15.63,62.0,H-SC,8.12,257638.0,-23485.0,156335.0,-1.27,-8.35,60.68,47.26,144.0,-0.15,1.88,38.04,XY24,NTT,INSURANCE
50,PGHH,17907.65,-26.58,59.0,X-MC,2.90,212415.0,11595.0,56205.0,-0.13,5.77,26.46,33.76,80.0,0.21,1.55,11.27,X40,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,44.04
2,50,75.84


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.36
LC    33.71
MC    21.92
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.77
X40      14.83
XY25     12.09
XR       10.99
X40N     10.08
AR        8.18
OX40N     7.70
X200      1.81
X5K       1.60
SR        1.12
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.64
X-LC    22.60
X-MC    16.12
M-SC    12.67
M-LC     5.36
H-LC     4.72
X-SC     4.61
H-MC     3.83
M-MC     1.66
L-SC     1.44
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.05,-6.55,41.66
IT,13.04,-17.48,81.10
FINANCE,12.15,-13.37,64.76
MISC,6.90,-21.12,88.24
BANKS,6.49,-12.16,71.15
PAINTS,5.61,-19.89,37.76
ELECTRICAL,5.34,-12.43,51.88
INSURANCE,3.87,-4.15,43.07
AUTO,2.84,-47.68,118.28


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3352959.0,22
XR,1400895.0,14
AR,1207416.0,9
X40,804079.0,10
X40N,643855.0,8
OX40N,590273.0,10
XY25,524350.0,8
SR,268000.0,2
X5K,98475.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3483419.0,25
M-SC,1302127.0,15
X-LC,1139071.0,13
X-MC,1115656.0,11
X-SC,547712.0,6
H-MC,354815.0,3
H-LC,292081.0,3
M-LC,273423.0,4
L-SC,263264.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1208699.0      6
           XR         831175.0      7
           AR         826087.0      5
M-SC       XY24       705004.0      6
X-MC       XY24       590808.0      4
X-LC       X40        497936.0      5
           XY24       308060.0      2
X-SC       X40N       302763.0      4
H-SC       OX40N      277332.0      4
           SR         268000.0      2
X-MC       X40        225883.0      4
X-LC       X40N       219643.0      3
H-LC       AR         215427.0      2
H-MC       XY24       185009.0      1
X-MC       XY25       177516.0      2
L-SC       XR         173087.0      2
M-SC       AR         165902.0      2
X-SC       XY24       164689.0      1
M-MC       XY24       164428.0      1
M-SC       OX40N      150065.0      4
           XY25       148228.0      1
           XR         132928.0      2
X-MC       X40N       121449.0      1
X-LC       XY25       113432.0      3
M-LC       XR         106400.0      1
           X5K         98475.0      1
H-MC       XR          97107.0      1
L-SC       OX40N       90177.0      1
X-SC       X40         80260.0      1
H-LC       X200        76654.0      1
H-MC       OX40N       72699.0      1
H-SC       MH          72126.0      1
L-MC       XR          60198.0      1
L-LC       XY25        42888.0      1
M-LC       XY25        42286.0      1
           XY24        26262.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
